In [2]:
import os
import pandas as pd
import matplotlib as plt

In [3]:
df = pd.read_csv('total_run_1_parsed_outputs.csv')
print(df)

      INDEX        TRIAL          PROMPT                    MODEL  \
0         0  total_run_1  prompt_prompt1             gemma2-9b-it   
1         1  total_run_1  prompt_prompt1             gemma2-9b-it   
2         2  total_run_1  prompt_prompt1             gemma2-9b-it   
3         3  total_run_1  prompt_prompt1             gemma2-9b-it   
4         4  total_run_1  prompt_prompt1             gemma2-9b-it   
...     ...          ...             ...                      ...   
2368   2368  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   
2369   2369  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   
2370   2370  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   
2371   2371  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   
2372   2372  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   

           SOURCE DATA_ID            FILENAME  VULN_MODEL  VULN_TRUTH  
0     cwe_samples    1004   1004_Bad.java.txt        True        True  
1     cwe_samples    1004  

In [4]:
countCorrect = ((df['VULN_MODEL']) == (df['VULN_TRUTH'])).sum()
print(f'Correct: {countCorrect}')
countFalse = ((df['VULN_MODEL']) != (df['VULN_TRUTH'])).sum()
print(f'Mismatched: {countFalse}')

Correct: 1931
Mismatched: 442


In [7]:
df_llama = df[df["MODEL"] == "llama-3.1-70b-versatile" ]
countCorrect = ((df_llama['VULN_MODEL']) == (df_llama['VULN_TRUTH'])).sum()
print(f'Correct: {countCorrect}')
countFalse = ((df_llama['VULN_MODEL']) != (df_llama['VULN_TRUTH'])).sum()
print(f'Mismatched: {countFalse}')
print(f'Total: {len(df_llama)}')
print(f"Score: {countCorrect/len(df_llama)*100}%")

Correct: 969
Mismatched: 220
Total: 1189
Score: 81.49705634987384%


In [8]:
df_gemma = df[df["MODEL"] == "gemma2-9b-it" ]
countCorrect = ((df_gemma['VULN_MODEL']) == (df_gemma['VULN_TRUTH'])).sum()
print(f'Correct: {countCorrect}')
countFalse = ((df_gemma['VULN_MODEL']) != (df_gemma['VULN_TRUTH'])).sum()
print(f'Mismatched: {countFalse}')
print(f'Total: {len(df_gemma)}')
print(f"Score: {countCorrect/len(df_gemma)*100}%")

Correct: 962
Mismatched: 222
Total: 1184
Score: 81.25%
